In [31]:
import pandas as pd
from IPython.display import clear_output
path_output = 'data/results/all.gff'
path_blast = 'data/results/'
pd.set_option('display.max_colwidth', -1)

In [32]:
#TEs
params = {}
#params['LTR'] = {'min_len':650,'max_len':False,'min_distance':5,'max_q':1.4,'min_q':0.6,'min_pident':60,'min_qcov':80,'file':''}
params['LINE'] = {'min_len':1500,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':60,'min_qcov':80,'file':'line_clustered.blast.csv.filtered'}
params['SINE'] = {'min_len':150,'max_len':800,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':'sine_clustered.blast.csv.filtered'}
params['TIR'] = {'min_len':500,'max_len':False,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':90,'min_qcov':90,'file':'tirs_clustered.blast.csv.filtered'}
params['MITE'] = {'min_len':50,'max_len':800,'min_distance':5,'max_q':1.15,'min_q':0.85,'min_pident':90,'min_qcov':90,'file':'mites_clustered.blast.csv.filtered'}
params['helitrons'] = {'min_len':2000,'max_len':False,'min_distance':5,'max_q':1.2,'min_q':0.8,'min_pident':60,'min_qcov':80,'file':'helitrons_clustered.blast.csv.filtered'}


In [33]:
#read blast output
dfs = []
for k,v in params.items():
    df = pd.read_csv(path_blast + v['file'], sep='\t')
    dfs.append(df)
df_all = pd.concat(dfs)
df_all.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
df_all = df_all.reset_index(drop=True)
print('initial:',len(df_all.index))

('initial:', 50381)


In [34]:
df_all['source'] = 'ST_annotation'
df_all['frame'] = '.'
df_all['score'] = '.'
df_all['strand'] = '+'
df_all['feature'] = 'TE'
df_all['qseqid'] = df_all['qseqid'].str.split('[', 1).str[1]
df_all['qseqid'] = df_all['qseqid'].str.replace("\\]\\[",";")
df_all['qseqid'] = df_all['qseqid'].str.replace("]","")
df_all['qseqid'] = df_all['qseqid'].str.replace("[","")
df_all = df_all[['sseqid','source','feature','sstart','send','score','strand','frame','qseqid']]
#gff3 -> ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

In [35]:
#order by chromosome and position
df_all['new_sstart'] = df_all[['sstart','send']].min(axis=1)
df_all['new_ssend'] = df_all[['sstart','send']].max(axis=1)
df_all['sstart'] = df_all['new_sstart']
df_all['send'] = df_all['new_ssend']
df_all = df_all.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df_all.sort_values(['sseqid', 'sstart'], inplace=True)

In [36]:
#delete dups annotation
for k,v in df_all.iterrows():
    qs = {}
    for q in v.qseqid.split(";"):
        qs[q] = 1
    new_qseqid = ";".join(qs.keys())
    df_all.at[k, 'qseqid'] = new_qseqid

TypeError: _get_value() takes at least 3 arguments (3 given)

In [30]:
df_all.to_csv(path_output, sep="\t", header=None, index=None)
path_output

'data/results/all.gff'